In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../data/pl_clean.csv')

X = data.drop('result', axis=1)
y = data['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Draw', 'Loss', 'Win']))

[[0 1 3]
 [2 3 0]
 [0 0 5]]
              precision    recall  f1-score   support

        Draw       0.00      0.00      0.00         4
        Loss       0.75      0.60      0.67         5
         Win       0.62      1.00      0.77         5

    accuracy                           0.57        14
   macro avg       0.46      0.53      0.48        14
weighted avg       0.49      0.57      0.51        14

